In [5]:
import pandas as pd
import numpy as np
import os
from datetime import datetime
import datetime as dt

In [6]:
STOCK_LIST = os.listdir(r'./1_min_datasets/')
LOT_SIZES_DF = pd.read_csv(r'./lot_sizes.csv')
LOT_SIZES_LIST = list(LOT_SIZES_DF.iloc[:, 1])

In [7]:
len(LOT_SIZES_LIST)

188

In [8]:
stock_counter = 0

for STOCK, LOT_SIZE in zip(STOCK_LIST[0:1], LOT_SIZES_LIST[0:1]):
    df = pd.read_csv(f'./1_min_datasets/{STOCK}')

    SLOPE_THRESHOLD_BUY_LIST = [0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8]
    SLOPE_THRESHOLD_SELL_LIST = [-0.2, -0.3, -0.4, -0.5, -0.6, -0.7, -0.8]
    LOOKBACK_PERIOD_LIST = list(range(5, 15))
    # SLOPE_THRESHOLD_BUY_LIST = [0.7]
    # SLOPE_THRESHOLD_SELL_LIST = [-0.2]
    # LOOKBACK_PERIOD_LIST = [6]
    COST_PER_TRANSACTION = 105
    COMBINATIONS = len(SLOPE_THRESHOLD_BUY_LIST) * len(SLOPE_THRESHOLD_SELL_LIST) * len(LOOKBACK_PERIOD_LIST)
    COLUMNS=['buy_threshold', 'sell_threshold', 'lookback_period', 'profitable_transactions', 'loss_transactions', 'total_transactions', 'total_transaction_cost', 'adjusted_P&L', 'day_1', 'day_2', 'day_3', 'day_4', 'day_5', 'day_6', 'day_7', 'day_8', 'day_9', 'day_10', 'day_11', 'day_12', 'day_13', 'day_14', 'day_15', 'day_16', 'day_17', 'day_18', 'day_19', 'day_20', 'day_21']
    STOCK_NAME = STOCK.split('.')[0]

    df['Date'] = pd.to_datetime(df['Date'])
    dates_arr = np.array([x.date() for x in df['Date']])

    if len(np.unique(dates_arr)) != 21:
        print(f'Missing Data for {STOCK}')
        print('Moving to next stock')
        continue

    df_track = pd.DataFrame(columns=COLUMNS)

    row_count = 0

    # print(f'{stock_counter}/{len(STOCK_LIST)} \t {STOCK_NAME} \t {LOT_SIZE}')
    stock_counter += 1

    for SLOPE_THRESHOLD_BUY in SLOPE_THRESHOLD_BUY_LIST:

        for SLOPE_THRESHOLD_SELL in SLOPE_THRESHOLD_SELL_LIST:
        
            for LOOKBACK_PERIOD in LOOKBACK_PERIOD_LIST:
                
                position = 0
                buy_price = sell_price = np.nan
                profit_loss_daily = []
                # n_trades = 0
                daily_trades = 0
                daily_trades_count = []
                profitable_transactions = 0
                loss_transactions = 0
                profit_loss = 0


                for i in range(LOOKBACK_PERIOD, len(df.loc[:, 'Close'])):

                    x = np.array(list(range(0, LOOKBACK_PERIOD)))
                    price_list = np.array(df.loc[i - LOOKBACK_PERIOD : i - 1, 'Close'])
                    N = len(x)
                    slope = (N * np.sum(x * price_list) - np.sum(x) * np.sum(price_list)) / ((N * np.sum(x ** 2)) - (np.sum(x) ** 2))
                    date_i = df['Date'][i]

                    # Current Time: 9.15 - 3.15
                    if date_i.time() < dt.time(15, 15):

                        # Position = 0
                        if position == 0:
                            if slope > SLOPE_THRESHOLD_BUY:
                                position = 1
                                # n_trades += 1
                                daily_trades += 1
                                buy_price = price_list[-1]
                                action = f'DateTime: {date_i.time()} \t Position: 0 --> 1 \t Buy Price: {buy_price} \t Slope: {slope}'
                                # print(action)
                            elif slope < SLOPE_THRESHOLD_SELL:
                                position = -1
                                # n_trades += 1
                                daily_trades += 1
                                sell_price = price_list[-1]
                                action = f'DateTime: {date_i.time()} \t Position: 0 --> -1 \t Sell Price: {sell_price} \t Slope: {slope}'
                                # print(action)

                        # Position = 1 (LONG)
                        if position == 1:
                            if slope < 0:
                                position = 0
                                # n_trades += 1
                                daily_trades += 1
                                sell_price = price_list[-1]
                                action = f'DateTime: {date_i.time()} \t Position: 1 --> 0 \t Sell Price: {sell_price} \t Slope: {slope}'
                                # print(action)
                                # P&L Calculation:
                                profit_loss += sell_price - buy_price
                            else:
                                # action = f'DateTime: {date_i} \t Position: 1 --> 1 \t HOLD'
                                pass

                        if position == -1:
                            if slope > 0:
                                position = 0
                                # n_trades += 1
                                daily_trades += 1
                                buy_price = price_list[-1]
                                action = f'DateTime: {date_i.time()} \t Position: -1 --> 0 \t Buy Price: {buy_price} \t Slope: {slope}'
                                # print(action)
                                # P&L Calculation:
                                profit_loss += sell_price - buy_price
                            else:
                                # action = f'DateTime: {date_i} \t Position: -1 --> -1 \t HOLD'
                                pass

                        
                    if date_i.time() < dt.time(15, 28):
                        
                        if position == 1:
                            if slope < 0:
                                position = 0
                                # n_trades += 1
                                daily_trades += 1
                                sell_price = price_list[-1]
                                action = f'DateTime: {date_i.time()} \t Position: 1 --> 0 \t Sell Price: {sell_price} \t Slope: {slope}'
                                # print(action)
                                # P&L Calculation:
                                profit_loss += sell_price - buy_price
                            else:
                                # action = f'DateTime: {date_i} \t Position: 1 --> 1 \t HOLD'
                                pass
                        elif position == -1:
                            if slope > 0:
                                position = 0
                                # n_trades += 1
                                daily_trades += 1
                                buy_price = price_list[-1]
                                action = f'DateTime: {date_i.time()} \t Position: -1 --> 0 \t Buy Price: {buy_price} \t Slope: {slope}'
                                # print(action)
                                # P&L Calculation:
                                profit_loss += sell_price - buy_price
                            else:
                                # action = f'DateTime: {date_i} \t Position: -1 --> -1 \t HOLD'
                                pass
                        else: # position = 0
                            # action = f'DateTime: {date_i} No new positions to be taken after 3.15'
                            pass

                    if date_i.time() == dt.time(15, 28):

                        if position == 1:
                            position = 0
                            # n_trades += 1
                            daily_trades += 1
                            sell_price = price_list[-1]
                            action = f'DateTime: {date_i.time()} \t Position: 1 --> 0 \t Sell Price: {sell_price} \t Slope: {slope}'
                            # print(action)
                            # P&L Calculation:
                            profit_loss += sell_price - buy_price

                        elif position == -1:
                            position = 0
                            # n_trades += 1
                            daily_trades += 1
                            buy_price = price_list[-1]
                            action = f'DateTime: {date_i.time()} \t Position: -1 --> 0 \t Buy Price: {buy_price} \t Slope: {slope}'
                            # print(action)
                            # P&L Calculation:
                            profit_loss += sell_price - buy_price

                        else: # position = 0
                            # action = f'DateTime: {date_i} No new positions to be taken after 3.15'
                            pass

                        daily_trades_count.append(daily_trades)
                        profit_loss_daily.append(profit_loss)
                        # profit_loss = 0
                        # daily_trades = 0

                total_transaction_cost = sum(daily_trades_count) * COST_PER_TRANSACTION
                adjusted_profit_loss = (sum(profit_loss_daily) * LOT_SIZE) - total_transaction_cost
                adjusted_profit_loss_daily = [x * LOT_SIZE - y * COST_PER_TRANSACTION for x, y in zip(profit_loss_daily, daily_trades_count)]

                df_track.loc[row_count, 'buy_threshold'] = SLOPE_THRESHOLD_BUY
                df_track.loc[row_count, 'sell_threshold'] = SLOPE_THRESHOLD_SELL
                df_track.loc[row_count, 'lookback_period'] = LOOKBACK_PERIOD
                df_track.loc[row_count, 'profitable_transactions'] = profitable_transactions
                df_track.loc[row_count, 'loss_transactions'] = loss_transactions
                df_track.loc[row_count, 'total_transactions'] = sum(daily_trades_count)
                df_track.loc[row_count, 'total_transaction_cost'] = total_transaction_cost
                df_track.loc[row_count, 'adjusted_P&L'] = adjusted_profit_loss
                df_track.iloc[row_count, 8:] = adjusted_profit_loss_daily

                row_count += 1
            
                # print(f'------------------------------------------------------------------------------')
                # print(f'Combination: {row_count}')
                # print(f'Threshold High: {SLOPE_THRESHOLD_BUY}, Threshold Low: {SLOPE_THRESHOLD_SELL}, Lookback Period: {LOOKBACK_PERIOD}')
                # print(f'Total trades = {sum(daily_trades_count)}')
                # print(f'Total cost of trades = {total_transaction_cost}')
                # print(f'Adjusted P&L for {STOCK} having {LOT_SIZE} LOT Size = {adjusted_profit_loss:.2f}')
                # print(f'Profitable Trades: {profitable_transactions}')
                # print(f'Loss Trades: {loss_transactions}')

    df_track_partial = df_track[(df_track['adjusted_P&L'] > 0) & (df_track['total_transactions'] <= 126)].copy().sort_values(by=['adjusted_P&L'], ascending=False)
    df_track.sort_values(by='adjusted_P&L', ascending=False, inplace=True)
    
    df_track_partial.to_csv(f'./regressor_results_partial_new/{STOCK_NAME}_regressor_partial.csv', index=False)
    df_track.to_csv(f'./regressor_results_complete_new/{STOCK_NAME}_regressor_complete.csv', index=False)


KeyboardInterrupt: 

In [ ]:
df_track

,buy_threshold,sell_threshold,lookback_period,profitable_transactions,loss_transactions,total_transactions,total_transaction_cost,adjusted_P&L,day_1,day_2,...,day_12,day_13,day_14,day_15,day_16,day_17,day_18,day_19,day_20,day_21
351,0.7,-0.2,6,0,0,40,4200,5192.5,5192.5,5192.5,...,5192.5,5192.5,5192.5,5192.5,5192.5,5192.5,5192.5,5192.5,5192.5,5192.5
374,0.7,-0.4,9,0,0,14,1470,4905.0,4905.0,4905.0,...,4905.0,4905.0,4905.0,4905.0,4905.0,4905.0,4905.0,4905.0,4905.0,4905.0
444,0.8,-0.4,9,0,0,14,1470,4905.0,4905.0,4905.0,...,4905.0,4905.0,4905.0,4905.0,4905.0,4905.0,4905.0,4905.0,4905.0,4905.0
304,0.6,-0.4,9,0,0,14,1470,4735.0,4735.0,4735.0,...,4735.0,4735.0,4735.0,4735.0,4735.0,4735.0,4735.0,4735.0,4735.0,4735.0
452,0.8,-0.5,7,0,0,14,1470,4395.0,4395.0,4395.0,...,4395.0,4395.0,4395.0,4395.0,4395.0,4395.0,4395.0,4395.0,4395.0,4395.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51,0.2,-0.7,6,0,0,40,4200,-17290.0,-17290.0,-17290.0,...,-17290.0,-17290.0,-17290.0,-17290.0,-17290.0,-17290.0,-17290.0,-17290.0,-17290.0,-17290.0
20,0.2,-0.4,5,0,0,64,6720,-17770.0,-17770.0,-17770.0,...,-17770.0,-17770.0,-17770.0,-17770.0,-17770.0,-17770.0,-17770.0,-17770.0,-17770.0,-17770.0
40,0.2,-0.6,5,0,0,54,5670,-17825.0,-17825.0,-17825.0,...,-17825.0,-17825.0,-17825.0,-17825.0,-17825.0,-17825.0,-17825.0,-17825.0,-17825.0,-17825.0
60,0.2,-0.8,5,0,0,48,5040,-18682.5,-18682.5,-18682.5,...,-18682.5,-18682.5,-18682.5,-18682.5,-18682.5,-18682.5,-18682.5,-18682.5,-18682.5,-18682.5
